## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

## And so on and so forth...

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt
from moviepy.editor import VideoFileClip

# Step-1 Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

In [2]:
ret, mtx, dist, revcs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [3]:
#Step-2 Undistort: apply a distortion correction to raw images.
def undistortImg(image):
    img_size = (image.shape[1], image.shape[0])
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    return undist

images = glob.glob('../test_images/test*.jpg')  
#images = glob.glob('../camera_cal/calibration*.jpg')
show_flag = True
for image in images:
    img = mpimg.imread(image)
    if show_flag == True:
        undistort = undistortImg(img)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 12))
        ax1.imshow(img)
        ax1.set_title("Original img")
        ax2.imshow(undistort)
        ax2.set_title("Undistorted img")
        

In [4]:
#Step-3-1 Use color transforms, gradients, etc., to create a thresholded binary image. 
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    # Return the result
    return binary_output

# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    # Return the binary image
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output


In [5]:
# Step-3-2 Binary thresholds
# Convert from the warped image to binary threshold images.
def binaryThresholding(img): 
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize = 3) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    thresh_min = 20
    thresh_max = 100
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Threshold saturation channel
    s_thresh_min = 170
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    binary = np.zeros_like(s_channel)
    binary[(s_binary == 1) | (sxbinary == 1)] = 1
    color_binary = np.dstack((binary, binary, binary))
    return color_binary

In [6]:
#Step-4 Apply a perspective transform to rectify binary image ("birds-eye view").
def birdsEye(img, show_flag = False):
    # Apply gaussian to reduce the noise
    undist = undistortImg(img)
    undist = cv2.GaussianBlur(undist, (5, 5), 0)
    binary_img = binaryThresholding(undist)
    img_size = (undist.shape[1], undist.shape[0])
    offset = 0
    
    # in the top-left, top-right, bottom-right, and bottom-left order
    # For source points I'm grabbing the outer four detected corners    
   
    src = np.float32([[560, 460],[720, 460],
                      [1215, 720],[64, 720]])
    dst = np.float32([[128, 0], [1152, 0], 
                     [1152, 720],[128, 720]])
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(binary_img, M, img_size, flags = cv2.INTER_LINEAR)
    
    if show_flag == True:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 12))
        ax1.imshow(img)
        for i in range(4):
            ax1.plot(src[i][0],src[i][1],'r*')
        ax1.set_title("Original img")
        ax2.imshow(warped)
        ax2.set_title("Birds_eye img")
        
    return warped, M, M_inv

show_flag = True
for image in images:
    img = mpimg.imread(image)
    birdsEye(img, show_flag)
    

In [7]:
# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

In [8]:
# Step-5 find the lane pixels
def findLanes(binary_warped):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped, axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base 
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,1,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,1,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    # Fit a second order polynomial to each pixel positions
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    y_eval = np.max(ploty)
        
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [1, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 1]
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)

    return out_img, ploty, left_fit, left_fitx, leftx_base, right_fit, right_fitx, rightx_base, left_curverad, right_curverad

In [9]:
test_img_name = "../test_images/test2.jpg"
test_img = mpimg.imread(test_img_name)

In [10]:
binary_warped, M, M_inv = birdsEye(test_img, True)
plt.imshow(binary_warped)
plt.show()
#lanes
out_img, ploty, left_fit, left_fitx, leftx_base, right_fit, right_fitx, rightx_base, left_curverad, right_curverad = findLanes(binary_warped[:,:,0])

In [33]:
# Determine the curvature of the lane and vehicle position with respect to center.
def drawImg(image, warped, ploty, leftx_base, left_fit, left_fitx, rightx_base, right_fit, right_fitx, Minv, left_curverad, right_curverad):
    font = cv2.FONT_HERSHEY_DUPLEX
    color_warp = np.zeros_like(warped).astype(np.uint8)
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (warped.shape[1], warped.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)

    #location_string = carLocation(warped, leftx_base, rightx_base)
    #----Add radius of curvature based on real world space rather than pixel space

    # Now our radius of curvature is in meters
    curvature_str = "Left curvature is " + str(int(left_curverad)) + "m, and right curvature is " + str(int(right_curverad)) + "m"   
    print(left_curverad, 'm', right_curverad, 'm')
    #-------------------------------
 
    # offset set to meter
    pos = warped.shape[1]/2
    offset = (pos - (leftx_base + rightx_base)/2) * xm_per_pix
    if offset > 0:
        location_str = "Vehicle is {:.2f}m left from center".format(offset)
    else:
        location_str = "Vehicle is {:.2f}m right from center".format(abs(offset))


        
    cv2.putText(result, curvature_str,(300,50), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(result, location_str,(300,100), font, 1,(255,255,255),2,cv2.LINE_AA)

    #cv2.imwrite('/Users/lei/Udacity/writeup/Advanced_lane_findings/result.png', result)
    return result


In [34]:
result = drawImg(test_img, binary_warped, ploty, leftx_base, left_fit, left_fitx, rightx_base, right_fit, right_fitx, M_inv, left_curverad, right_curverad)

596.730092083 m 482.94928388 m


In [35]:
def processVideo(img):
    binary_warped, M, M_inv = birdsEye(img)
    #lanes
    out_img, ploty, left_fit, left_fitx, leftx_base, right_fit, right_fitx, rightx_base, left_curverad, right_curverad = findLanes(binary_warped[:,:,0])
    result = drawImg(img, binary_warped, ploty, leftx_base, left_fit, left_fitx, rightx_base, right_fit, right_fitx, M_inv, left_curverad, right_curverad)
    return result

In [36]:
video_output = './result.mp4'
clip1 = VideoFileClip("../project_video.mp4")
white_clip = clip1.fl_image(processVideo) 
white_clip.write_videofile(video_output, audio=False)

502.547904298 m 771.850890401 m
[MoviePy] >>>> Building video ./result.mp4
[MoviePy] Writing video ./result.mp4


  0%|          | 1/1261 [00:00<05:31,  3.80it/s]

502.547904298 m 771.850890401 m


  0%|          | 2/1261 [00:00<05:26,  3.86it/s]

490.015775368 m 897.555040304 m


...

100%|█████████▉| 1259/1261 [05:58<00:00,  3.31it/s]

19129.0980051 m 1055.61552479 m


100%|█████████▉| 1260/1261 [05:58<00:00,  3.34it/s]

6597.84045173 m 1336.53725747 m


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./result.mp4 

